In [14]:
import pandas as pd
import numpy as np
import pickle
import re
import nltk
from nltk.corpus import stopwords
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 1. KONFIGURASI
MAX_LEN = 120
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

print("Memuat aset model...")

# 2. LOAD MODEL & TOKENIZER
try:
    model = load_model('model_cnn_clash_royale.keras')
    print("✅ Model berhasil dimuat!")
except:
    print("❌ Error: File 'model_cnn_clash_royale.keras' tidak ditemukan.")

try:
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)
    print("✅ Tokenizer berhasil dimuat!")
except:
    print("❌ Error: File 'tokenizer.pickle' tidak ditemukan.")

# 3. FUNGSI PREPROCESSING
def clean_text_inference(text):
    text = str(text).lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text) 
    words = text.split()
    words = [w for w in words if w not in stop_words]
    return " ".join(words)

# 4. PREDIKSI
def prediksi_sentimen(text):
    clean = clean_text_inference(text)

    seq = tokenizer.texts_to_sequences([clean])

    padded = pad_sequences(seq, maxlen=MAX_LEN)
    
    pred = model.predict(padded, verbose=0)
    label_idx = np.argmax(pred)
    confidence = np.max(pred) * 100
    
    labels = {0: 'Negatif', 1: 'Netral', 2: 'Positif'}
    
    return labels[label_idx], confidence

# 5. TESTING MANUAL
print("\n=== UJI COBA MODEL 2 (CNN 1D) ===")

reviews_baru = [
    "Worst Game I ever played, full of bugs and pay-to-win mechanics.", 
    "Best strategy game ever! I really enjoy the clan wars.",           
    "Totally trash, matchmaking is unfair and server lags.",            
    "It is okay, good for passing time."                                
]

for text in reviews_baru:
    label, conf = prediksi_sentimen(text)
    print(f"Review   : {text}")
    print(f"Prediksi : {label} (Yakin: {conf:.2f}%)")
    print("-" * 40)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Memuat aset model...
✅ Model berhasil dimuat!
✅ Tokenizer berhasil dimuat!

=== UJI COBA MODEL 2 (CNN 1D) ===
Review   : Worst Game I ever played, full of bugs and pay-to-win mechanics.
Prediksi : Negatif (Yakin: 100.00%)
----------------------------------------
Review   : Best strategy game ever! I really enjoy the clan wars.
Prediksi : Positif (Yakin: 100.00%)
----------------------------------------
Review   : Totally trash, matchmaking is unfair and server lags.
Prediksi : Negatif (Yakin: 100.00%)
----------------------------------------
Review   : It is okay, good for passing time.
Prediksi : Positif (Yakin: 99.98%)
----------------------------------------
